In [ ]:
LANE DETECTION PROJECT

In [ ]:
#importing packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidYellowLeft.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image) 

In [ ]:
import math

def colorSelection(img):
    """
    Removes all colors except white and yellow.
    """
    #Convert to HLS color space to improve colorSelection accuarcy
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    #Set color thresholds
    white_lower = np.array([0, 200, 0])
    white_upper = np.array([255, 255, 255])
    yellow_lower = np.array([10, 0, 100])
    yellow_upper = np.array([40, 255, 255])
    
    #define color masks
    white_mask = cv2.inRange(hls, white_lower, white_upper)
    yellow_mask = cv2.inRange(hls, yellow_lower, yellow_upper)
    
    #Apply masks to image
    final_mask = cv2.bitwise_or(white_mask, yellow_mask)
    return cv2.bitwise_and(hls, hls, mask=final_mask)

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    #return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=20):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    left_lines = []
    left_length = []
    right_lines = []
    right_length = []
                
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                if x2==x1:
                    continue #ignore vertical lines
                slope = ((y2 - y1) / (x2 - x1))
                intercept = y1 - slope * x1 # y = mx + b
                length = int(np.sqrt((y2 - y1)**2 + (x2 - x1)**2)) #A2 + B2 = C2
                
                if slope < -0.5: #negative slope = left lane
                    left_lines.append((slope, intercept))
                    left_length.append((length))
                elif slope > 0.5: #positive slope = right lane
                    right_lines.append((slope, intercept))
                    right_length.append((length))     
                else:
                    continue #ignore lines with less than 0.5 slope
          

        #average lines slope and intercept
        #longer lines have more weight
        left_lane = np.dot(left_length, left_lines) / np.sum(left_length)
        right_lane = np.dot(right_length, right_lines) / np.sum(right_length)
        
        lanes = left_lane, right_lane
        
        for lane in lanes:
            if isinstance(lane, np.ndarray):
                slope, intercept = lane

                #Convert slope to coordinates
                y1 = int(img.shape[0] + 20)
                y2 = int(img.shape[0] * 0.65)
                x1 = int((y1 - intercept) / slope)
                x2 = int((y2 - intercept) / slope)

                #Draw line
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)
        

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [ ]:
import os
os.listdir("test_images/")

In [ ]:
#Start by isolating yellow and white pixels
colorSelect = colorSelection(image)

#Convert to grayscale
gray = grayscale(colorSelect)

#Next apply Gaussian smoothing
kernel_size = 7
blur_gray = gaussian_blur(gray, kernel_size)

#Run Canny Edge Detection
low_threshold = 50
high_threshold = 150
edges = canny(blur_gray, low_threshold, high_threshold)

#Apply image mask to refine edges
imshape = image.shape
max_y = imshape[0]
max_x = imshape[1]

vertices = np.array([[(max_x * 0.4, max_y * 0.6), (max_x * 0.1, max_y), (max_x * 0.95, max_y), (max_x * 0.6, max_y * 0.6)]] ,dtype=np.int32)
masked_edges = region_of_interest(edges, vertices)

#Hough transform to draw lines from points generated in canny edge detection
rho = 1
theta = np.pi/180
threshold = 35
min_line_len = 15
max_line_gap = 25
lines = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)

#Overlay Hough lines on original image
result = weighted_img(lines, image, α=0.8, β=1., γ=0.)


plt.imshow(result)
#plt.imshow(masked_edges, cmap='gray')

#mpimg.imsave("test_images_output/solidYellowLeftMaskedEdges.png", masked_edges, cmap='gray')

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    #Start by isolating yellow and white pixels
    colorSelect = colorSelection(image)

    #Convert to grayscale
    gray = grayscale(colorSelect)

    #Next apply Gaussian smoothing
    kernel_size = 7
    blur_gray = gaussian_blur(gray, kernel_size)

    #Run Canny Edge Detection
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    #Apply image mask to refine edges
    imshape = image.shape
    max_y = imshape[0]
    max_x = imshape[1]

    vertices = np.array([[(max_x * 0.4, max_y * 0.6), (max_x * 0.1, max_y), (max_x * 0.95, max_y), (max_x * 0.6, max_y * 0.6)]] ,dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)

    #Hough transform to draw lines from points generated in canny edge detection
    rho = 1
    theta = np.pi/180
    threshold = 35
    min_line_len = 15
    max_line_gap = 25
    lines = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)

    #Overlay Hough lines on original image
    result = weighted_img(lines, image, α=0.8, β=1., γ=0.)
    
    return result

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(3,7)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))